## Introduction

This notebook uses Julia to collect data from **interest.co.nz** & **www.ofx.com** and save data as csv files. The data including:
- Official Cash Rate
- CPI
- Currency Exchange Rate
  - NZD-USD
  - NZD-RMB

In [7]:
using Pkg
using HTTP
using JSON
using Dates
using DataFrames
using CSV

dir = @__DIR__

"/Users/zhoujinze/Documents/DataScience/ds-assessment/DATA-422/group project/data422-group-project"

## Functions to Collect Data

In [22]:
function save_csv(data::DataFrame, filename)
    path = dir * "/data/" * filename
    CSV.write(path, data)
end

function get_json(id::String)
    resp = HTTP.request("POST", "https://www.interest.co.nz/chart-data/get-csv-data";
        headers = Dict("Content-Type" => "application/x-www-form-urlencoded; charset=utf-8"),
        body = HTTP.URIs.escapeuri(Dict("nids[]" => id)))

    json_body = JSON.parse(transcode(String, resp.body))
    return json_body
end

function json2dataframe(id::String, json, colname)
    lines = json[id]["csv_data"][1]

    df = DataFrame(date = String[], data = Float64[])

    for line in lines
        t = unix2datetime(line[1] / 1000)
        ts = Dates.format(t, "Y-m-d")
        d = line[2]
        push!(df, [ts, d])
    end

    rename!(df,:data => colname)

end

function get_save_csv(id::String, filename, colname)
    json = get_json(id)
    df = json2dataframe(id, json, colname)
    save_csv(df, filename)
end

get_save_csv (generic function with 1 method)

## Data

In [33]:
# HPI data
get_save_csv("79394", "hpi.csv", "HPI")

"/Users/zhoujinze/Documents/DataScience/ds-assessment/DATA-422/group project/data422-group-project/data/hpi.csv"

In [23]:
# official cash rate
get_save_csv("79430", "official_cash_rate.csv", "ocr")

"/Users/zhoujinze/Documents/DataScience/ds-assessment/DATA-422/group project/data422-group-project/data/official_cash_rate.csv"

In [25]:
# CPI of Newzealand
get_save_csv("70951", "cpi_nz.csv", "CPI")

"/Users/zhoujinze/Documents/DataScience/ds-assessment/DATA-422/group project/data422-group-project/data/cpi_nz.csv"

In [27]:
# nzd-usd echange rate
get_save_csv("79333", "nzd-usd_exchange_rate.csv", "USD exchange rate")

"/Users/zhoujinze/Documents/DataScience/ds-assessment/DATA-422/group project/data422-group-project/data/nzd-usd_exchange_rate.csv"

We extract NZD-RMB data from **www.ofx.com**.

In [31]:
# URL: https://api.ofx.com/PublicSite.ApiService//SpotRateHistory/allTime/NZD/CNY?DecimalPlaces=6&ReportingInterval=monthly&format=json
resp = HTTP.request("GET", "https://api.ofx.com/PublicSite.ApiService//SpotRateHistory/allTime/NZD/CNY";
    query = Dict("DecimalPlaces" => "6", "ReportingInterval" => "monthly", "format" => "json"))

nzd_cny_json =  JSON.parse(transcode(String, resp.body))
nc_his = nzd_cny_json["HistoricalPoints"]

nc_df = DataFrame(date = String[], NZD_CNY_rate = Float64[], CNY_NZD_rate = Float64[])

for nc in nc_his
    d =  unix2datetime(nc["PointInTime"]/1000)
    dt = Dates.format(d, "Y-m-d")
    ncr = nc["InterbankRate"]
    cnr = nc["InverseInterbankRate"]
    push!(nc_df, [dt, ncr, cnr])
end

save_csv(nc_df, "nzd-cny_exchange_rate.csv")


"/Users/zhoujinze/Documents/DataScience/ds-assessment/DATA-422/group project/data422-group-project/data/nzd-cny_exchange_rate.csv"